In [10]:
import cv2
import numpy as np

# Load image
img = cv2.imread("people.jpg")
if img is None:
    raise FileNotFoundError("Image not found. Place 'sample.jpg' in this folder.")

clone = img.copy()
rect_start = None
rect_end = None
dragging = False
moving = False
roi = None
offset = (0, 0)

# Transparency factor
alpha = 0.6  # 0 = fully transparent, 1 = fully opaque

def end_move(x, y, alpha_img, alpha_roi):
    global rect_start, rect_end, dragging, moving, roi, offset, img, clone
    
    # Move ROI with transparency
    img = clone.copy()
    dx = x - offset[0]
    dy = y - offset[1]
    offset = (x, y)
    new_rect = (rect_start[0] + dx, rect_start[1] + dy,
                rect_end[0] + dx, rect_end[1] + dy)

    x1, y1, x2, y2 = new_rect
    # Clamp coordinates to image boundaries
    h, w = img.shape[:2]
    x1 = max(0, min(x1, w - roi.shape[1]))
    y1 = max(0, min(y1, h - roi.shape[0]))
    x2, y2 = x1 + roi.shape[1], y1 + roi.shape[0]

    # Blend ROI with background
    roi_resized = roi.copy()
    blended = cv2.addWeighted(img[y1:y2, x1:x2], alpha_img, roi_resized, alpha_roi, 0)

    img[y1:y2, x1:x2] = blended
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    rect_start = (x1, y1)
    rect_end = (x2, y2)

def mouse_events(event, x, y, flags, param):
    global rect_start, rect_end, dragging, moving, roi, offset, img, clone

    if event == cv2.EVENT_LBUTTONDOWN:
        # Start drawing rectangle
        rect_start = (x, y)
        dragging = True
        roi = None
        img = clone.copy()

    elif event == cv2.EVENT_MOUSEMOVE and dragging:
        # Draw rectangle while dragging
        img = clone.copy()
        cv2.rectangle(img, rect_start, (x, y), (0, 255, 0), 2)

    elif event == cv2.EVENT_LBUTTONUP:
        # Finish drawing ROI
        rect_end = (x, y)
        dragging = False
        x1, y1 = rect_start
        x2, y2 = rect_end
        roi = clone[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)].copy()
        cv2.rectangle(img, rect_start, rect_end, (0, 255, 0), 2)

    elif event == cv2.EVENT_RBUTTONDOWN and roi is not None:
        # Start moving
        moving = True
        offset = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE and moving:
        # Move ROI with transparency
        end_move(x, y, 1-alpha, alpha)

    elif event == cv2.EVENT_RBUTTONUP:
        moving = False
        end_move(x, y, 0, 1)

# Create window
cv2.namedWindow("Move ROI with Transparency")
cv2.setMouseCallback("Move ROI with Transparency", mouse_events)

while True:
    cv2.imshow("Move ROI with Transparency", img)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC to exit
        break

cv2.destroyAllWindows()